# 10. 앙상블 학습

여러 약한 분류기를 결합하는 AdaBoost를 구현합니다.

## 학습 목표
- 앙상블 학습의 원리 이해
- AdaBoost 알고리즘 이해
- 약한 분류기와 강한 분류기

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)

## 1. 이진 분류 데이터 생성

In [ ]:
# 2D 이진 분류 데이터
n_samples = 200

# 클래스 1
X1 = torch.randn(n_samples // 2, 2) * 0.5 + torch.tensor([1.0, 1.0])
# 클래스 -1
X2 = torch.randn(n_samples // 2, 2) * 0.5 + torch.tensor([-1.0, -1.0])

X = torch.cat([X1, X2], dim=0)
y = torch.cat([torch.ones(n_samples // 2), -torch.ones(n_samples // 2)])

# 섞기
perm = torch.randperm(n_samples)
X, y = X[perm], y[perm]

plt.figure(figsize=(8, 6))
plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', label='Class +1', alpha=0.7)
plt.scatter(X[y == -1, 0], X[y == -1, 1], c='red', label='Class -1', alpha=0.7)
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Binary Classification Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. 약한 분류기: Decision Stump

결정 그루터기(Decision Stump)는 단일 특징과 임계값으로 분류하는 가장 간단한 분류기입니다.

In [ ]:
from mlfs.classical.ensemble import DecisionStump

# 단일 Decision Stump
stump = DecisionStump()
stump.fit(X, y)

print(f"Feature: {stump.feature_idx}")
print(f"Threshold: {stump.threshold:.4f}")
print(f"Polarity: {stump.polarity}")

# 예측
pred = stump.predict(X)
accuracy = (pred == y).float().mean()
print(f"Accuracy: {accuracy:.4f}")

## 3. AdaBoost 알고리즘

1. 모든 샘플에 동일한 가중치 초기화
2. 각 라운드에서:
   - 가중치 기반으로 약한 분류기 학습
   - 가중 에러 계산
   - 분류기 가중치(α) 계산
   - 샘플 가중치 업데이트 (잘못 분류된 샘플 가중치 증가)

In [ ]:
from mlfs.classical.ensemble import AdaBoost

# AdaBoost 학습
ada = AdaBoost(n_estimators=10)
ada.fit(X, y)

print(f"Number of stumps: {len(ada.stumps)}")
print(f"\nStump weights (alpha):")
for i, alpha in enumerate(ada.alphas[:5]):
    print(f"  Stump {i+1}: {alpha:.4f}")

In [ ]:
# 정확도 확인
pred = ada.predict(X)
accuracy = (pred == y).float().mean()
print(f"AdaBoost Accuracy: {accuracy:.4f}")

## 4. 결정 경계 시각화

In [ ]:
def plot_decision_boundary(model, X, y, title):
    """결정 경계 시각화"""
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    
    xx, yy = torch.meshgrid(
        torch.linspace(x_min, x_max, 100),
        torch.linspace(y_min, y_max, 100),
        indexing='ij'
    )
    
    grid = torch.stack([xx.flatten(), yy.flatten()], dim=1)
    Z = model.predict(grid).reshape(xx.shape)
    
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='RdBu')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', label='Class +1', edgecolors='k')
    plt.scatter(X[y == -1, 0], X[y == -1, 1], c='red', label='Class -1', edgecolors='k')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title(title)
    plt.legend()
    plt.show()

# 단일 Stump vs AdaBoost
plot_decision_boundary(stump, X, y, 'Single Decision Stump')
plot_decision_boundary(ada, X, y, 'AdaBoost (10 Stumps)')

## 5. 에스티메이터 수에 따른 성능

In [ ]:
# 훈련/테스트 분할
n_train = int(0.8 * n_samples)
X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

n_estimators_list = [1, 5, 10, 20, 50, 100]
train_accs = []
test_accs = []

for n_est in n_estimators_list:
    model = AdaBoost(n_estimators=n_est)
    model.fit(X_train, y_train)
    
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    train_accs.append((train_pred == y_train).float().mean().item())
    test_accs.append((test_pred == y_test).float().mean().item())

plt.figure(figsize=(10, 6))
plt.plot(n_estimators_list, train_accs, 'b-o', label='Train')
plt.plot(n_estimators_list, test_accs, 'r-o', label='Test')
plt.xlabel('Number of Estimators')
plt.ylabel('Accuracy')
plt.title('AdaBoost: Effect of Number of Estimators')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. 더 복잡한 데이터에서의 AdaBoost

In [ ]:
# XOR 패턴 데이터
n = 100
X_xor = torch.cat([
    torch.randn(n, 2) * 0.3 + torch.tensor([0.5, 0.5]),
    torch.randn(n, 2) * 0.3 + torch.tensor([-0.5, -0.5]),
    torch.randn(n, 2) * 0.3 + torch.tensor([0.5, -0.5]),
    torch.randn(n, 2) * 0.3 + torch.tensor([-0.5, 0.5])
], dim=0)

y_xor = torch.cat([
    torch.ones(n), torch.ones(n), -torch.ones(n), -torch.ones(n)
])

# AdaBoost with many stumps
ada_xor = AdaBoost(n_estimators=50)
ada_xor.fit(X_xor, y_xor)

plot_decision_boundary(ada_xor, X_xor, y_xor, 'AdaBoost on XOR Pattern (50 Stumps)')

pred_xor = ada_xor.predict(X_xor)
print(f"XOR Accuracy: {(pred_xor == y_xor).float().mean():.4f}")

## 요약

1. **앙상블 학습**: 여러 약한 분류기를 결합하여 강한 분류기 생성
2. **AdaBoost**: 순차적으로 학습, 잘못 분류된 샘플에 집중
3. **Decision Stump**: 가장 간단한 약한 분류기 (단일 임계값)
4. **장점**: 과적합에 강함, 복잡한 경계 학습 가능
5. **핵심 수식**: α = 0.5 * log((1-ε)/ε), 가중치 업데이트

다음: **얼굴 검출** (Haar-like 특징, Viola-Jones)